In [1]:
import pandas as pd
from IPython.display import Image, HTML
from datetime import datetime

In [2]:
# convert link to html tags 
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [3]:
dfs = pd.read_csv("inputs/dfs.csv")
fd = pd.read_csv("inputs/fanduel.csv")
dk = pd.read_csv("inputs/draftkings.csv")
st = pd.read_csv("inputs/statline.csv")
dfs = dfs.drop("Unnamed: 0", axis=1)
fd = fd.drop("Unnamed: 0", axis=1)
dk = dk.drop("Unnamed: 0", axis=1)
st = st.drop("Unnamed: 0", axis=1)

In [4]:
ts = datetime.now()
ts = ts.strftime("%b %d, %Y %I:%M %p")
ts = "Last Updated: " + ts

In [5]:
def highlight_dfs(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*19
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*19
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*19
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*19
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*19
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*19
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*19
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*19
    else: return ['background-color: white']*19

In [10]:
def highlight_fd(df):
    if df.Hot > 0.0 and df.Hot < 0.5: return ['background-color: #ff9999']*14
    elif df.Hot >= 0.5 and df.Hot < 1.0: return ['background-color: #ff6666']*14
    elif df.Hot >= 1.0 and df.Hot < 1.5: return ['background-color: #ff3333']*14
    elif df.Hot >= 1.5: return ['background-color: #ff0000']*14
    elif df.Cold > 0.0 and df.Cold < 0.5: return ['background-color: #99e6ff']*14
    elif df.Cold >= 0.5 and df.Cold < 1.0: return ['background-color: #66d9ff']*14
    elif df.Cold >= 1.0 and df.Cold < 1.5: return ['background-color: #33ccff']*14
    elif df.Cold >= 1.5: return ['background-color: #00bfff']*14
    else: return ['background-color: white']*14

In [7]:
def gen_page(df, out, coloring):
    games = pd.read_csv('games.csv').loc[:,['name','slug']]
    trans_games = games.set_index('name').T
    img_dict = trans_games.to_dict('list')
    images = []
    for i in df["Name"]:
        code = "https://thumbs.dreamstime.com/b/creative-illustration-default-avatar-profile-placeholder-isolated-background-art-design-grey-photo-blank-template-mockup-144849704.jpg"
        if img_dict.get(i) != None:
            code = "https://d2cwpp38twqe55.cloudfront.net/req/202005142/images/players/" + img_dict.get(i)[0] + ".jpg"
        images.append(code)
    images = ['<img src="' + image + '" width="60">' for image in images]
    df["Picture"] = images
    
    cols = df.columns.tolist()
    cols = [cols[0]] + cols[-1:] + cols[1:-1]
    df = df[cols]
    pd.set_option('display.max_colwidth', -1)
    df.index +=1
    pre = (
        "<head>\n<script src=\"https://kryogenix.org/code/browser/sorttable/sorttable.js\"></script>\n"
        "<link rel=\"stylesheet\" href=\"df_styles.css\">\n</head>\n"
    )
    pre = pre + ts + "\n"
    pre += (
        "\n<body>\n<div id=\"filter\">\n<input type=\"text\" id=\"name\" onkeyup=\"nameSearch()\" placeholder=\"Search for name...\">\n"
        "<input type=\"text\" id=\"team\" onkeyup=\"teamSearch()\" placeholder=\"Search for team...\">\n"
        "<input type=\"text\" id=\"position\" onkeyup=\"posSearch()\" placeholder=\"Search for position...\">\n"
        "</div>\n\n"
    )
    
    post = (
        "\n"
        "\n<script src=\"main.js\"></script>"
        "\n</body>"
        )
    # render dataframe as html
    if coloring is not None:
        body = df.style.apply(coloring, axis=1).render()
    else:
        html = HTML(df.to_html(escape=False ,formatters=dict(Picture=path_to_image_html)))
        body = html.data

    body = body.replace("dataframe", "sortable", 1)
    body = body.replace("class=\"sortable\"", "id=\"tbl\" class=\"sortable\"", 1)
    a = pre + body + post
    # write html to file
    with open(out, 'w', encoding="utf-8") as f:
        f.write(a)

In [11]:
gen_page(dfs, 'dfs.html', highlight_dfs)
gen_page(fd, 'fd.html', highlight_fd)
gen_page(dk, 'dk.html', None)
gen_page(st, 'st.html', None)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
